## Instruction
In this notebook, We use data which I donwload from Tushare to make up some alpha factors.
1. Load data which time range bettwen 2018.1 - 2023.3. download from privious step `DownLoad_from_Tushare.ipynb` 
2. Make up some factors based on some paper, tech indicators and fundamentls

## Load Data
load data which already download from Tushare

In [1]:
import pandas as pd 
import numpy as np
from tqdm import tqdm
from scipy.stats import zscore, spearmanr, rankdata

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [9, 5]

from alpha_191 import Alpha191

import warnings
warnings.filterwarnings('ignore')

In [28]:
universe_raw = pd.read_csv('tushare_data/raw_119.csv').iloc[:,1:]
fundamental_df = pd.read_csv('tushare_data/fundamental_119.csv').iloc[:,1:]
universe_raw = universe_raw.drop_duplicates(subset=['ts_code','trade_date']).reset_index(drop=True)

In [ ]:
universe = universe_raw.rename(columns={'vol':'volume'})
universe = universe.drop_duplicates(subset=['ts_code','trade_date'])
universe.head()

In [ ]:
print(len(universe.ts_code.unique()), universe.shape)
print(len(fundamental_df.ts_code.unique()))
print(len(universe.columns), universe.columns)

## Data Explain
  Some data columns about fundamental indicators below:
  
  - 'ts_code','ann_date',
  - 'cfps','revenue_ps','gross_margin', # 每股现金流，每股营业收入, 毛利
  - 'dt_eps','dt_eps_yoy',              # 每股收益， 每股收同比益增长率
  - 'bps','bps_yoy',                    #每股净资产，每股净资产同比增长率
  - 'extra_item','profit_dedt',         # 扣非，扣非净利润
  - 'roe_dt','roe_yoy',                 #净资产收益，净资产收益同比增长率
  - 'ebt_yoy','or_yoy'                  # 总利润增长率，营业收入增长率
  
  If there are some columns named alpha begin, the data had processed completed. Each column named alpha_ is alpha factors

### Filter Custom Tickers Pool

In [ ]:
# remove tickers by each day not exist history
def remove_tickers(df, exist_ticker_list):
    diff_df = df.loc[df.ts_code.isin(exist_ticker_list)==False]
    if diff_df.empty == False:
        # remove tickers not exist over 3month from day
        diff1 = (pd.to_datetime(diff_df['trade_date'],format='%Y%m%d')\
                -pd.to_datetime(diff_df['list_date'],format='%Y%m%d')).dt.days < 22
        
        # remove 
        # remove liquid few than 5 millon
        diff2 = diff_df['amount']<5000
        
        diff3 = diff_df['total_share'] > 100000
        #diff4 = diff_df['close'] > 30
        diff5 = diff_df['dt_eps'] <= 0.4
        diff6 = diff_df['pe'] > 35
        diff7 = diff_df['pb'] > 4
        diff8 = diff_df['total_mv'] > 2000000
        #diff8 = diff_df['revenue_ps']>0
        
        # get remove df
        diff_df = diff_df.loc[diff1| diff2| diff3| diff6| diff7| diff8]
        # get rest data
        df = df.loc[df.ts_code.isin(diff_df.ts_code)==False]
    return df

calendar = universe.trade_date.unique()
df_all = pd.DataFrame()
for dt in tqdm(calendar, desc='filter tikers'):
    tmp = universe.loc[universe['trade_date']==dt]
    if df_all.empty:
        tmp = remove_tickers(tmp, [])
    else:
        tmp = remove_tickers(tmp, df_all.ts_code.unique())
    df_all = df_all.append(tmp, ignore_index=True)
universe = df_all
universe['date'] = pd.to_datetime(universe['trade_date'], format='%Y%m%d')
universe = universe.set_index(['date']).sort_values(by=['date'])
# update stock list
#fundamental_df = fundamental_df.loc[fundamental_df.ts_code.isin(universe.ts_code.unique())]
print(universe.shape, len(universe.ts_code.unique()))

## Add Technology Indicators

In [ ]:
import stockstats

class IndicatorHelper(pd.DataFrame):
    """
        add indicators to dataframe
    """

    def __init__(self, data):
        super(IndicatorHelper, self).__init__(data)

        self.stocks = stockstats.StockDataFrame.retype(data.copy())
        self.df = self

    def add_technical_indicator(self, tech_indicator_list):
        """
        calculate technical indicators
        use stockstats package to add technical inidactors
        :param ticker: (df) pandas dataframe
        :param tech_indeicator_list list
        :return: (df) pandas dataframe
        """
        unique_ticker = self.df.ts_code.unique()

        indicator_df = pd.DataFrame()
        for i in tqdm(range(len(unique_ticker)), desc='add tech indicators'):
            temp_indicator = self.stocks[self.stocks.ts_code == unique_ticker[i]]
            temp_indicator = temp_indicator[tech_indicator_list + ['ts_code','trade_date']]
            indicator_df = indicator_df.append(temp_indicator, ignore_index=True)
        
        self.df = self.df.merge(indicator_df, on=["ts_code", "trade_date"], how="left")
        self.df['date'] = pd.to_datetime(self.df['trade_date'],format='%Y%m%d')
        self.df = self.df.set_index(['date']).sort_values(by=['date']).drop_duplicates(['trade_date','ts_code'])
        return self.df
    

In [ ]:
# add tech indicators
tech_indicator_list = ['log-ret','atr_6', 'atr_14', 'ppo', 'close_10_kama_5_30', 'close_10_kama_2_30'] 
universe = IndicatorHelper(universe).add_technical_indicator(tech_indicator_list)
# 'close_2_kama_20_mstd'

In [ ]:
def alpha_kama(df):
    feature = 'close_10_kama_5_30'
    def cal_(df):
        kama_filter = df[feature].rolling(14).std()
        cond_in1 = (df[feature] - df[feature].shift(1)) >kama_filter
        cond_in2 = (df[feature].shift(1) - df[feature].shift(2)) > kama_filter
        cond_out1 = (df[feature] - df[feature].shift(1)) < -kama_filter
        cond_out2 = (df[feature].shift(1) - df[feature].shift(2)) < -kama_filter
        df['alpha_kama'] = np.where((cond_out1 & cond_out2), -df['close'] , - 0.3*kama_filter)
        df['alpha_kama'] = np.where((cond_in1 & cond_in2), df['close'] , df['alpha_kama'])
        #df['alpha_kama'] = np.where((cond_out1 & cond_out2), df['log-ret']-1, 0)
        #df['alpha_kama'] = np.where((cond_in1 & cond_in2), df['log-ret']+1, df['alpha_kama'])
        
        return df
    
    df_all = pd.DataFrame()
    for ts_code in tqdm(df.ts_code.unique(), desc='alpha_kama processing...'):
        tmp = df.loc[df.ts_code == ts_code]
        tmp = cal_(tmp)
        df_all = df_all.append(tmp)
    
    return df_all.sort_values(by=['date'])

#universe = universe.drop(columns=['alpha_kama'])
universe = alpha_kama(universe)

### Indicator Factors

In [ ]:
universe['vwap'] = universe['amount']/universe['volume']*10
universe['alpha_ppo'] = -universe['ppo']
universe['alpha_atr'] = -universe['atr_6']
universe['alpha_atr14'] = -universe['atr_14']
universe['alpha_pb'] = -universe['pb']

### Construct Factors Based on Alpha 191

In [4]:
#universe_bak = universe.copy(deep=True)
alpha = Alpha191(universe, None)

100%|████████████████████████████████████████████████████████████████████| 1295/1295 [00:03<00:00, 326.93it/s]


In [ ]:
universe.columns

In [ ]:
universe = alpha.alpha007(alpha.df) #1

In [ ]:
universe = alpha.alpha009(alpha.df) #1

In [ ]:
universe = alpha.alpha010(alpha.df)

In [ ]:
universe = alpha.alpha021(alpha.df) #1

In [ ]:
universe = alpha.alpha026(alpha.df) #1

In [ ]:
universe = alpha.alpha029(alpha.df) #1

In [ ]:
universe = alpha.alpha056(alpha.df) #1

In [ ]:
universe = alpha.alpha069(alpha.df) #1

In [ ]:
universe = alpha.alpha075(universe) #1

In [9]:
universe = alpha.alpha083(alpha.df) 

In [10]:
universe = alpha.alpha101(alpha.df) #1

processing...: 100%|████████████████████████████████████████████████████████| 119/119 [03:15<00:00,  1.64s/it]


In [11]:
universe = alpha.alpha103(alpha.df) 

In [12]:
universe = alpha.alpha111(alpha.df) #1

In [13]:
universe = alpha.alpha116(alpha.df)

processing...: 100%|████████████████████████████████████████████████████████| 119/119 [00:29<00:00,  4.01it/s]


In [ ]:
universe = alpha.alpha122(alpha.df) #1

In [ ]:
universe = alpha.alpha190(alpha.df) #1

In [5]:
universe = alpha.alpha149(alpha.df) #1

In [6]:
universe = alpha.alpha150(alpha.df) #1

In [7]:
universe = alpha.alpha030(alpha.df) #1

alpha030 processing...: 100%|███████████████████████████████████████████████| 119/119 [20:51<00:00, 10.51s/it]


In [14]:
universe = alpha.alpha176(alpha.df)

processing...: 100%|████████████████████████████████████████████████████████| 119/119 [01:43<00:00,  1.15it/s]


In [15]:
universe = alpha.alpha190(alpha.df)

In [8]:
def alpha_kama_revert(df):
    def cal_(df):
        df['alpha_kama2'] = -df['close_10_kama_2_30'].rolling(6).std()
        return df
    df = df.groupby('ts_code').apply(cal_)
    return df

universe = universe.sort_values(by=['date','ts_code'])
universe = alpha_kama_revert(universe)

In [ ]:
universe['alpha_131'] = -universe['alpha_021'] * universe['alpha_116']

In [ ]:
## tmp bak
alpha.df = universe.copy(deep=True)
bak = universe.copy(deep=True)

In [3]:
universe.columns

Index(['ts_code', 'trade_date', 'turnover_rate', 'pe', 'pb', 'ps', 'dv_ratio',
       'total_share', 'free_share', 'total_mv', 'circ_mv', 'name', 'industry',
       'list_date', 'pct_change', 'volume', 'amount', 'open', 'close', 'high',
       'low', 'pre_close', 'cfps', 'revenue_ps', 'gross_margin', 'dt_eps',
       'dt_eps_yoy', 'bps', 'bps_yoy', 'extra_item', 'profit_dedt', 'roe_dt',
       'roe_yoy', 'ebt_yoy', 'or_yoy', 'log-ret', 'atr_6', 'atr_14', 'ppo',
       'close_10_kama_5_30', 'close_10_kama_2_30', 'alpha_kama', 'vwap',
       'alpha_ppo', 'alpha_atr', 'alpha_atr14', 'alpha_pb', 'rsize', 'rvalue',
       'alpha_007', 'alpha_009', 'alpha_010', 'alpha_021', 'alpha_026',
       'alpha_029', 'alpha_056', 'alpha_069', 'alpha_075'],
      dtype='object')

In [37]:
ticker_list = ['688363', '603102', '603368', '002462', '000411', '002788', '600976', '600211', '301311', '002864',
        '300878', '002728', '600750', '300519', '002873', '301047', '688358', '688314', '301033', '300396',
        '688317', '600587', '300918', '300463', '300639', '300677', '600056', '300642', '300363', '300109',
        
        '300294', '603590', '688163', '300485', '001367', '000915', '603351', '300233', '603229', '300723',
        '003020', '688799', '688117', '002940', '301130', '603538', '605177', '002923', '300583', '000756',
        '002393', '603811', '300636', '002365', '600513', '600062', '300584', '301075', '002826', '300016',]
ticker_list = [int(_) for _ in ticker_list]
tmp_save = universe_raw.rename(columns={'vol':'volume'})
tmp_save = tmp_save.drop_duplicates(subset=['ts_code','trade_date'])

In [39]:
df_list = fundamental_df.loc[fundamental_df.symbol.isin(ticker_list)]
tmp_save = tmp_save.loc[tmp_save.ts_code.isin(df_list.ts_code.unique())]

In [40]:
tmp_save

,ts_code,trade_date,turnover_rate,pe,pb,ps,dv_ratio,total_share,free_share,total_mv,...,dt_eps,dt_eps_yoy,bps,bps_yoy,extra_item,profit_dedt,roe_dt,roe_yoy,ebt_yoy,or_yoy
2,000915.SZ,20180102,1.1511,36.6170,4.9338,4.7026,0.3722,23433.1485,18488.8117,7.264276e+05,...,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.000000e+00,-0.0000,-0.0000,-0.0000,-0.0000
3,300485.SZ,20180102,1.0308,27.1294,3.1347,11.3761,0.8559,24000.0000,8448.0000,7.010400e+05,...,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.000000e+00,-0.0000,-0.0000,-0.0000,-0.0000
7,002365.SZ,20180102,3.6849,108.3234,5.2277,12.2829,0.2911,19645.5000,13660.8875,6.748229e+05,...,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.000000e+00,-0.0000,-0.0000,-0.0000,-0.0000
9,002826.SZ,20180102,0.6447,50.9955,4.8878,6.2467,0.4032,18972.0000,7207.4681,2.823034e+05,...,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.000000e+00,-0.0000,-0.0000,-0.0000,-0.0000
10,600587.SH,20180102,0.3582,198.1901,2.1159,0.8207,0.2664,40642.8091,28651.9026,6.864570e+05,...,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.000000e+00,-0.0000,-0.0000,-0.0000,-0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116879,300463.SZ,20230505,1.0784,13.9026,1.5375,2.7276,2.9123,61246.9590,42289.3838,9.842386e+05,...,0.1908,-43.8658,10.4521,1.7216,-3255475.75,1.201292e+08,1.8926,-45.2055,-45.7050,-26.8539
116880,603102.SH,20230505,3.8457,21.4044,2.0114,4.0677,1.0846,6400.0000,3029.1366,2.950400e+05,...,0.7000,32.0755,22.9193,3.1495,1120150.57,4.366307e+07,3.0228,32.2518,43.5712,58.7297
116881,300485.SZ,20230505,0.9864,25.6908,1.5588,7.1606,1.5385,48166.6400,20098.9984,5.259797e+05,...,0.0017,-99.2315,7.0054,0.0114,-2309052.99,3.132989e+06,0.0929,-99.2320,-97.9843,-30.9212
116884,600976.SH,20230505,2.0362,25.1531,5.2194,2.8158,1.2031,15339.8600,11028.5185,1.025163e+06,...,0.6500,12.0690,12.8042,5.4373,9127795.18,9.043232e+07,4.7260,-5.6164,12.7279,10.3976


In [41]:
tmp_save.to_csv('factor_data/all_60.csv')

In [16]:
universe.to_csv('factor_data/all_119_test.csv')

In [ ]:
universe = universe.sort_values(by=['trade_date', 'ts_code'])

## Rank Alpha Factors
As computer use factor to compare each oter before all factors had same scales. So we rescale each factors by Rank in each day and each industry. 

Here we pick some features as mechine learning step in the next step.

In [ ]:
sector_list = ['医药商业', '中成药', '化学制药', '医疗保健', '生物制药', '化工原料']
universe['ind_val'] = np.where(universe.industry.isin(sector_list), 1, 0)
universe['ind_val'].unique()

In [2]:
universe = pd.read_csv('factor_data/all_119_test.csv').iloc[:,1:]
universe['date'] = pd.to_datetime(universe['trade_date'],format='%Y%m%d')
universe = universe.set_index(['date']).sort_index(level=['date'])
#universe = universe.loc[universe['trade_date']>20210501]

In [ ]:
factor_names = [
            'alpha_kama', 'alpha_atr', 'alpha_atr14', 'alpha_kama2',
            'alpha_pb', 'alpha_ppo', 'alpha_fundamental', 'alpha_009', 'alpha_010',
            'alpha_056', 'alpha_069', 'alpha_075', 'alpha_083', 'alpha_101', 
            'alpha_103', 'alpha_122','alpha_149', 'alpha_150', 'alpha_030',
            'alpha_007', 'alpha_026', 'alpha_029', 'alpha_021', 'alpha_116', 'alpha_176', 'alpha_190'
        ]

In [ ]:
from scipy.stats import zscore

def rescale_(data, zscore_features=factor_names):
    data[factor_names] = data[factor_names].apply(zscore)
    # resize from -1 to 1
    for feature in zscore_features:
        #data[feature] = data[feature] - data[feature].median()
        max_val = data[feature].median() + 3.5*data[feature].std()
        min_val = data[feature].median() - 3.5*data[feature].std()
        data[feature] = np.where(data[feature]>max_val, max_val, 
                               np.where(data[feature]<min_val, min_val, data[feature]))
        #data[feature] = (data[feature] - data[feature].mean())/data[feature].std()
        data[feature] = np.where(data[feature]>0, data[feature]/data[feature].max(),
                                 np.where(data[feature]<0, data[feature]/(-data[feature].min()), 0.))
    return data

def zscore_by_sector(df, zscore_features):
    
    df_all = pd.DataFrame()
    for val in tqdm(df.ind_val.unique()):
        tmp = df.loc[df.ind_val == val]
        tmp[zscore_features] = tmp.groupby('trade_date')[zscore_features].apply(rescale_, zscore_features)
        df_all = df_all.append(tmp)

    return df_all.sort_values(by=['date'])

universe = universe.replace([np.inf, -np.inf], np.nan)
universe = universe.fillna(0.)
#universe = universe[~universe.isin([np.inf, -np.inf]).any(1)]

#universe = zscore_by_sector(universe, factor_names)
universe[factor_names] = universe.groupby('trade_date')[factor_names].apply(rescale_)

In [ ]:
#universe.loc[universe['trade_date']==20230417].sort_values(['alpha_atr14'])[['alpha_atr14','close','ts_code','name']]#.hist()
#universe['alpha_kama'].std()
universe.loc[universe['trade_date']==20230417].sort_values(['alpha_kama'])['alpha_111'].hist()


### Calculate Factor Positive Returns

In [ ]:
def factor_positive_profit(universe, features):
    df = universe.copy(deep=True)
    def process_(data):
        ## shift log-ret   
        data['p_ret4'] = data['log-ret'].shift(-2).fillna(0)
        data['p_ret5'] = data['log-ret'].shift(-3).fillna(0)
        data['p_ret6'] = data['log-ret'].shift(-4).fillna(0)
        ## only save factor positive values
        for feature in features:
            data[feature] = np.where(data[feature]>0, data[feature], 0)
        return data
    df = df.reset_index()
    df_all = pd.DataFrame()
    for ts_code in tqdm(df.ts_code.unique()):
        tmp = df.loc[df.ts_code==ts_code]
        df_all = df_all.append(process_(tmp))
    df = df_all.set_index(['date'])
    
    ## calculate factor return by shift returns
    for feature in features:
        df[feature + '_returns'] =  (df['p_ret4']  + df['p_ret5'] + df['p_ret6']) * df[feature]/3
    
    ## sum returns by each day
    p_ret_df = pd.DataFrame(index=df.index.unique())
    for dt in tqdm(p_ret_df.index, desc='cross sum feature returns'):
        tmp = df.loc[df.index==dt]
        for feature in features:
            p_ret_df.at[dt, feature] = tmp[feature + '_returns'].sum()/tmp[feature].sum()
        
    ## calculate positive factor cumsum returns
    p_ret_df.fillna(0)
    for feature in features:
        p_ret_df[feature] = p_ret_df[feature].cumsum()
            
    return p_ret_df

cal_features = [ 
            'alpha_atr14', 'alpha_kama', 'alpha_111', 'alpha_030',
            'alpha_028', 'alpha_035', 'alpha_061', 'close_10_kama_2_30', 'alpha_077', 'alpha_092',
            'alpha_098', 'alpha_172', 'alpha_190', 'alpha_112', 'alpha_128', 'alpha_131', 'alpha_kama2'
        ]
tmp_cal = universe.loc[universe['trade_date'] > 20220101]
positive_returns_df = factor_positive_profit(tmp_cal, cal_features)

In [ ]:
return_features = [
            'alpha_atr14', 'alpha_kama', 'alpha_111', 'alpha_kama2', 'alpha_030', 'close_10_kama_2_30', 'alpha_131', 
            #'alpha_028', 'alpha_035', 'alpha_061',   'alpha_077', 'alpha_092',
            #'alpha_098', 'alpha_172', 'alpha_190', 'alpha_112', 'alpha_128', 
        ]
positive_returns_df.loc[positive_returns_df.index>'2021-01-01'][return_features].plot(grid=True)

### Evaluate Alpha Factors

In [ ]:
#universe['close_10_kama_2_30'] = -universe['close_10_kama_2_30']
universe['alpha_131'] = -universe['alpha_131']

In [ ]:
all_factors = universe.copy(deep=True)[factor_names + ['trade_date','ts_code','industry', 'close', 'log-ret']]
all_factors.reset_index(drop=True, inplace=True)
all_factors['date'] = pd.to_datetime(all_factors['trade_date'], format='%Y%m%d')
all_factors = all_factors.set_index(['date','ts_code']).sort_values(by=['date'])
print(all_factors.shape)

In [ ]:
import alphalens as al
from scipy.stats import zscore
import helper.project_helper as ph

shift_day = -2
tmp = universe.copy(deep=True)
tmp['close'] = tmp.groupby('ts_code')['close'].shift(shift_day)
all_pricing = tmp.reset_index().pivot(index='date', columns='ts_code', values='close')
#all_pricing = all_pricing.loc[all_pricing.index[::5]]

def show_sample_results(factors_with_alpha, features, pricing=all_pricing, holding_time=1):
    factors_with_alpha = factors_with_alpha[features]
    
    # Setup data for AlphaLens
    print('Cleaning Data...\n')
    factor_data = ph.build_factor_data(factors_with_alpha, pricing, holding_time=holding_time)
    print('\n-----------------------\n')
    
    # Calculate Factor Returns and Sharpe Ratio
    factor_returns = ph.get_factor_returns(factor_data)
    sharpe_ratio = ph.sharpe_ratio(factor_returns)
    
    # Show Results
    print('             Sharpe Ratios')
    print(sharpe_ratio.round(2))
    ph.plot_factor_returns(factor_returns)
    ph.plot_factor_rank_autocorrelation(factor_data)
    
    qr_factor_returns = pd.DataFrame()
    for factor, factor_data in factor_data.items():
        qr_factor_returns[factor] = al.performance.mean_return_by_quantile(factor_data)[0]

    (10000*qr_factor_returns).plot.bar(
        subplots=True,
        sharey=True,
        layout=(5,2),
        figsize=(12, 8),
        legend=False)

#all_pricing.head()

In [ ]:
features = [
            'alpha_atr14', 'alpha_kama', 'alpha_kama2', 'alpha_030', 'alpha_128',
            'alpha_172', 'alpha_190', 'alpha_112', 'close_10_kama_2_30'
        ]
eva_df = all_factors.loc[all_factors['trade_date']>20220101]
show_sample_results(eva_df, features, pricing=all_pricing, holding_time=3)

In [ ]:
universe['alpha_021'] = -universe['alpha_021'] * universe['alpha_016']

### View factor to single ticker

In [ ]:
cond1 = universe['trade_date']>20230313
universe.loc[cond1].sort_values(by=['alpha_021'])[['ts_code','name','alpha_kama','alpha_021']].tail(10)

In [ ]:
#universe['alpha_021'].hist()
rescale_universe['alpha_cus021'].hist()

In [ ]:
name1 = 'alpha_021'
name2 = 'alpha_149'
# & (universe['trade_date']>20190901) 
rescale_universe = universe.copy()
#rescale_universe['alpha_021'] = rescale_universe['alpha_021'] * rescale_universe['alpha_atr']

# def alpha_021(df):
#     def cal_(data):
#         section1 = data['alpha_021'].diff(6).rolling(6).sum()
#         section2 = data['alpha_021'].shift(6)
#         cond1 = data['alpha_021'].diff(6)>0
#         cond2 = section1 > section2
#         data['alpha_cus021'] = np.where(cond1 & cond2, abs(data['alpha_021']), -abs(data['alpha_021']))
#         return data
#     df = df.groupby('ts_code').apply(cal_)
#     return df

# rescale_universe = alpha_021(rescale_universe)

# 'alpha_149'
cond1 = rescale_universe.ts_code=='300964.SZ'
cond2 = rescale_universe['trade_date']>20230101
cond3 = rescale_universe['trade_date']<20230502
tmp = rescale_universe.loc[cond1 & cond2&cond3][['close', name2, name1]]
#tmp[['close',name1, name2]] = tmp[['close', name1, name2]].apply(zscore)

tmp[['close']] = tmp[['close']].apply(zscore)
tmp['close'] = tmp['close'].shift(-2)
tmp.plot(title=name1, grid=True)

## Construct Factors Based on Paper

### 60days return

In [ ]:
def return_3month(df):
    def cal_(df):
        df['alpha_3m_ret'] = df['log-ret'].rolling(60).sum()
        return df
    df = df.groupby('ts_code').apply(cal_)
    return df.sort_values(by=['date'])

universe = return_3month(universe)

### atr custom

In [ ]:
universe['alpha_cus_atr'] = universe['atr_14']-universe['atr_6']

### Overnight Return

In [ ]:
def overnight_return(df):
    def cal_(df):
        section = (df['open'] - df['close'].shift(1).fillna(method='ffill')) / df['open']
        df['alpha_o2c'] = -section.rolling(50).sum()
        return df

    df = df.groupby('ts_code').apply(cal_)
    return df

#universe = universe.drop(columns=['alpha_o2c'])
universe = overnight_return(universe)

### Winner And Loser
This factor we also did in P4. It express a ticker how to reach a return in a spicific period time 

We use a time window as T, and regression d and v $return = T*d + T^2*v$  => $factor=d*v$ 

In [ ]:
from statsmodels.formula.api import ols

class WinnerAndLoser(pd.DataFrame):
    """
        Winner and Loser Factor Constructor
    """
    def __init__(self,data, win_length=14):
        super(WinnerAndLoser, self).__init__(data)
        self.df = self
        self.win_lenth = win_length

    def _regression(self, data):
        df = pd.DataFrame(data, columns=['log-ret'])
        df['acc_ret'] = df['log-ret'].cumsum()
        df['t_dir'] = np.arange(self.win_lenth)+1
        df['t_velocity'] = df['t_dir'] ** 2
        regression = ols(formula='acc_ret ~ 0 + t_dir + t_velocity', data=df)
        model = regression.fit()
        data['alpha_winlos'] = model.params.t_dir * model.params.t_velocity
        return  data['alpha_winlos']


    def calculate(self):
        '''
        convert time to value
        regress return to get mu and beta each time
        add facotor mu*beta to colomns
        :return: dataframe
        '''
        tickers = self.df.ts_code.unique()
        factor_df = pd.DataFrame()
        for ticker in tqdm(tickers, desc='winner and loser'):
            tmp_df = self.df.loc[self.df.ts_code == ticker][['trade_date', 'ts_code', 'log-ret']]
            tmp_df['alpha_winlos'] = tmp_df['log-ret'].rolling(self.win_lenth).apply(self._regression)
            tmp_df['alpha_winlos'].fillna(method='bfill',inplace=True)
            factor_df = factor_df.append(tmp_df, ignore_index=True)
        self.df = self.df.merge(factor_df[["ts_code", "trade_date", "alpha_winlos"]], on=["ts_code", "trade_date"], how="left")
        self.df['date'] = pd.to_datetime(self.df['trade_date'],format='%Y%m%d')
        self.df = self.df.set_index(['date']).sort_values(by=['date'])
        return  self.df
    
    
#test = universe.loc[universe.ts_code=='603538.SH']
#test = universe.loc[universe.ts_code=='002038.SZ']
#test = WinnerAndLoser(test).calculate()
#universe = universe.drop(columns=['alpha_winlos'])
universe = WinnerAndLoser(universe).calculate()

### Skew And Momentum
This factor we also did in P4. It express minority and majority sentiment of investor how to impact on market.

We calculate skew and median of log-return distribution in a period time, the skew view as marjority sentiment and median can view as minority sentiment.

$factor = skew * median 

In [ ]:
class SkewandMomentum(pd.DataFrame):
    """
        Expected Skewness and Momentum Factor Constructor
    """
    def __init__(self,data, win_length=14):
        super(SkewandMomentum, self).__init__(data)
        self.df = self
        self.win_length = win_length

    def calculate(self):
        '''
        convert time to value
        regress return to get mu and beta each time
        add facotor mu*beta to colomns
        :return: dataframe
        '''
        def calculate_factor(data):
            return data.skew() * data.median()
        
        tmp_df = pd.DataFrame()
        for stock_tuple in tqdm(self.groupby('ts_code'), desc='skew and momentum'):
            stock = stock_tuple[1]
            stock['alpha_skew2sentiment'] = stock['log-ret'].rolling(self.win_length).apply(calculate_factor)
            stock['alpha_skew2sentiment'] = stock['alpha_skew2sentiment'].fillna(method='bfill')
            tmp_df = tmp_df.append(stock,ignore_index=True)
        self.df = self.df.merge(tmp_df[["ts_code", "trade_date", "alpha_skew2sentiment"]], on=["ts_code", "trade_date"], how="left")
        self.df['date'] = pd.to_datetime(self.df['trade_date'],format='%Y%m%d')
        self.df = self.df.set_index(['date']).sort_values(by=['date'])
        return self.df
    
#universe = universe.drop(columns=['alpha_skew2sentiment'])
universe = SkewandMomentum(universe).calculate()

## Save All Data and factors to csv

In [ ]:
tmp_features = [
            'ts_code', 'trade_date', 'name', 'industry', 'close', 'log-ret', 'ind_val',
            'alpha_kama', 'alpha_atr14', 'alpha_083', 'alpha_103',  'alpha_111',
            'alpha_pb',  'alpha_101', 'alpha_atr', 'alpha_010',  'alpha_supertrend','alpha_021', 'alpha_056',
            'alpha_149', 'alpha_075', 'alpha_089','alpha_122', 'alpha_069'
            ]

universe_tmp = universe[tmp_features]
universe_tmp.to_csv('factor_data/all_108_zscore.csv')

In [ ]:
alpha.df.to_csv('factor_data/all_108.csv')

In [ ]:
df = pd.read_csv('factor_data/all_108.csv')
fundamental_df = pd.read_csv('tushare_data/fundamental_108.csv').iloc[:,1:]
df['date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')
df.set_index(['date'], inplace=True)

In [ ]:
universe = df.drop(columns=['rsize', 'rvalue', 'section1', 'section2'])
universe.columns

In [ ]:
df = pd.read_csv('factor_data/all_108_zscore.csv')
df.columns

### Load data fom csv

In [ ]:
ticker_list = [
    '603368', '002462', '600976', '600211', '002864', '300878', '002728', '600750', '002873', '688314',
    '301033', '688358', '688317', '600587', '300396', '300639', '300109', '300363', '603590', '000915',
    '300233', '603229', '300723', '003020', '688799', '688117', '603538', '002940', '605177', '002923',
    '300533', '300494', '603258', '603508', '301042', '300771', '300508', '300845', '300813', '688259',
    '603738', '688230', '003043', '688049', '688589', '300739', '300852', '300936', '605058', '603383',
    '301085', '688228', '688171', '688078', '300925', '605398', '300378', '002322', '300532', '300678',
    '002649', #'603102'
    ]
ticker_list = [int(_) for _ in ticker_list]

In [ ]:
fundamental_df = fundamental_df.loc[fundamental_df['symbol'].isin(ticker_list)]
df = df.loc[df.ts_code.isin(fundamental_df.ts_code.unique())]
print(fundamental_df.shape, len(df.ts_code.unique()))

In [ ]:
df.columns

In [ ]:
sector_list = ['医药商业', '中成药', '化学制药', '医疗保健', '生物制药']
universe['ind_val'] = np.where(universe.industry.isin(sector_list), 1, 0)
universe['ind_val'].unique()

In [ ]:
universe = universe.loc[universe.ind_val==1]
len(universe.ts_code.unique())

In [ ]:
universe.ts_code.unique()